In [860]:
import librosa
import numpy as np
from scipy.spatial.distance import cosine
from fastdtw import fastdtw
import noisereduce as nr
import simpledtw
import os

In [861]:
durasi = 12
sample_rate = 44100
target = durasi * sample_rate
def pad_signal(signal, target_len=target):
    
    num_zeros_needed = target_len - len(signal)
    
    if num_zeros_needed > 0:

        num_zeros_front = np.random.randint(num_zeros_needed)
        num_zeros_back = num_zeros_needed - num_zeros_front
        return np.pad(signal, (num_zeros_front, num_zeros_back), mode='constant')

    else:
        return signal

In [862]:
def pre_emphasis(signal):
    first_amp = signal[0]
    all_amps_without_first = signal[1:]
    all_amps_without_last = signal[:-1]
    emphasized_signal = np.append(first_amp, all_amps_without_first - 0.97 * all_amps_without_last)
    return emphasized_signal

In [863]:
def preprocessing(audio):
    y, sr = librosa.load(audio, sr=sample_rate)
    y, _ = librosa.effects.trim(y, top_db=20)
    y = librosa.util.normalize(y)
    y = nr.reduce_noise(y, sr)
    # y = pad_signal(y)
    return y, sr
    

In [864]:
def extraction(audio):
    y, sr = preprocessing(audio)
    # y = pre_emphasis(y)
    y = librosa.effects.preemphasis(y)

    mfccs = librosa.feature.mfcc(
        y = y,
        sr = sr,
        n_mfcc = 13,
        window = 'hamming',
        n_fft = int(sr * 0.025),
        hop_length = int(sr * 0.01),
        center = False,
        n_mels = 128,
        # lifter = 26,
        fmin=0,
        fmax=44100,
    )
    mfccs = librosa.util.normalize(mfccs)
    return mfccs.T

In [865]:
def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

In [866]:
def main(folder1, folder2, output_file):
    files1 = os.listdir(folder1)
    files2 = os.listdir(folder2)

    with open(output_file, 'w') as f:
        for file1 in files1:
            for file2 in files2:
                audio_path1 = os.path.join(folder1, file1)
                audio_path2 = os.path.join(folder2, file2)

                mfccs1 = extraction(audio_path1)
                mfccs2 = extraction(audio_path2)

                dtw_distance = calculate_dtw(mfccs1, mfccs2)

                result_line = f"Jarak DTW antara {file1} dan {file2}: {dtw_distance}\n"
                print(result_line)
                f.write(result_line)



folder1 = "../Dataset/03.Shad/"
folder2 = "../Dataset/03.Shad/"
output_file = '../hasil_perbandingan/tes.txt'

main(folder1, folder2, output_file)

Distance = 9.532363559197682
Distance = 18.557385489855328
Distance = 20.730941720840786
Distance = 28.241334316514823
Distance = 0


In [867]:
# audio_path1 = "../Dataset/01.Ha\'/Ha\'_M1_1.wav"
# audio_path2 = "../Dataset/01.Ha\'/Ha\'_M1_2.wav"
# audio_path3 = '../Dataset/09.Qaf/Qaf_M1_1.wav'
# audio_path4 = '../Dataset/09.Qaf/Qaf_M2_1.wav'
# audio_path5 = "../Dataset/01.Ha\'/Ha\'_M2_1.wav"


# mfccs1 = extraction(audio_path1)
# mfccs2 = extraction(audio_path2)
# mfccs3 = extraction(audio_path3)
# mfccs4 = extraction(audio_path4)
# mfccs5 = extraction(audio_path5)

# _, cost1, _, _, _ = simpledtw.dtw(mfccs1, mfccs2)
# _, cost2, _, _, _ = simpledtw.dtw(mfccs1, mfccs5)
# _, cost3, _, _, _ = simpledtw.dtw(mfccs1, mfccs3)
# _, cost4, _, _, _ = simpledtw.dtw(mfccs1, mfccs4)

# print(f"Distance = {cost1}")
# print(f"Distance = {cost2}")
# print(f"Distance = {cost3}")
# print(f"Distance = {cost4}")